## Neuromorphic Classifier - Gradio Demo

## Imports

In [1]:
!pip install gradio -q
!pip install snntorch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which

In [2]:
!wget "https://github.com/MatTheTab/neuromorphic_classifier/raw/refs/heads/main/models/neuromorphic_delta_model.pth"
!wget "https://github.com/MatTheTab/neuromorphic_classifier/raw/refs/heads/main/models/neuromorphic_rate_model.pth"
!wget "https://github.com/MatTheTab/neuromorphic_classifier/raw/refs/heads/main/models/neuromorphic_temporal_model.pth"

--2025-02-26 13:32:54--  https://github.com/MatTheTab/neuromorphic_classifier/raw/refs/heads/main/models/neuromorphic_delta_model.pth
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MatTheTab/neuromorphic_classifier/refs/heads/main/models/neuromorphic_delta_model.pth [following]
--2025-02-26 13:32:55--  https://raw.githubusercontent.com/MatTheTab/neuromorphic_classifier/refs/heads/main/models/neuromorphic_delta_model.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14506 (14K) [application/octet-stream]
Saving to: ‘neuromorphic_delta_model.pth’

neuromorphic_delta_ 100%[===================>] 

In [3]:
import gradio as gr
import random
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from IPython.display import HTML
import snntorch as snn
from snntorch import spikegen
from snntorch import surrogate
import torch.nn.functional as F
import snntorch.spikeplot as splt
import matplotlib.pyplot as plt
import os
import tempfile

## Variables

In [4]:
BATCH_SIZE = 64
IMAGE_REPITIONS = 4
EPOCHS = 30
SPIKE_GRAD = surrogate.fast_sigmoid(slope=25) #
BETA = 0.5
NUM_STEPS = 10
GAIN = 0.5
THRESHOLD_LATENCY = 1e-4
TAU = 0.9
THRESHOLD_DELTA = 0.5
NUM_CLASSES = 10

## Redefinitions

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)
dataiter = iter(trainloader)
images, labels = next(dataiter)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 485kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.45MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.82MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
class Neuromorphic_Net(nn.Module):
    '''
    A spiking neural network (SNN) model using leaky integrate-and-fire (LIF) neurons.

    Parameters:
    input_shape (tuple): Shape of the input data (batch_size, channels, height, width).
    num_classes (int): Number of output classes for classification.

    Attributes:
    conv1 (nn.Conv2d): First convolutional layer.
    lif1 (snn.Leaky): First LIF layer.
    conv2 (nn.Conv2d): Second convolutional layer.
    lif2 (snn.Leaky): Second LIF layer.
    conv3 (nn.Conv2d): Third convolutional layer.
    lif3 (snn.Leaky): Third LIF layer.
    output (nn.Linear): Fully connected output layer.
    lif4 (snn.Leaky): Fourth LIF layer.

    Methods:
    get_covnolution_output(): Computes the flattened output size after convolutional layers.
    reset_states(): Resets the membrane potentials of LIF layers.
    forward(x): Performs a forward pass through the network.

    Returns:
    torch.Tensor: Output spike activity of the final layer.
    '''
    def __init__(self, input_shape, num_classes):
        super().__init__()
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.in_channels = input_shape[1]
        self.conv1 = nn.Conv2d(self.in_channels, 4, 3)
        self.lif1 = snn.Leaky(beta=BETA, spike_grad=SPIKE_GRAD)
        self.conv2 = nn.Conv2d(4, 8, 3)
        self.lif2 = snn.Leaky(beta=BETA, spike_grad=SPIKE_GRAD)
        self.conv3 = nn.Conv2d(8, 16, 3)
        self.lif3 = snn.Leaky(beta=BETA, spike_grad=SPIKE_GRAD)
        self.output = nn.Linear(self.get_covnolution_output(), self.num_classes)
        self.lif4 = snn.Leaky(beta=BETA, spike_grad=SPIKE_GRAD)

    def get_covnolution_output(self):
        dummy_input = torch.rand(*self.input_shape)
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()
        cur1 = F.max_pool2d(self.conv1(dummy_input), 2)
        spk1, mem1 = self.lif1(cur1, mem1)
        cur2 = F.max_pool2d(self.conv2(spk1), 2)
        spk2, mem2 = self.lif2(cur2, mem2)
        cur3 = F.max_pool2d(self.conv3(spk2), 2)
        spk3, mem3 = self.lif3(cur3, mem3)
        return spk3.view(BATCH_SIZE, -1).shape[-1]

    def reset_states(self):
        self.mem1 = self.lif1.init_leaky()
        self.mem2 = self.lif2.init_leaky()
        self.mem3 = self.lif3.init_leaky()
        self.mem4 = self.lif4.init_leaky()

    def forward(self, x):
        cur1 = F.max_pool2d(self.conv1(x), 2)
        spk1, self.mem1 = self.lif1(cur1, self.mem1)
        cur2 = F.max_pool2d(self.conv2(spk1), 2)
        spk2, self.mem2 = self.lif2(cur2, self.mem2)
        cur3 = F.max_pool2d(self.conv3(spk2), 2)
        spk3, self.mem3 = self.lif3(cur3, self.mem3)
        output = self.output(spk3.view(spk3.shape[0], -1))
        spk4, self.mem4 = self.lif4(output, self.mem4)
        return spk4

In [7]:
def encode_rate(input):
    '''
    Encodes input data into spike trains using rate coding.

    Parameters:
    input (torch.Tensor): Input tensor to be encoded.

    Returns:
    torch.Tensor: Rate-encoded spike train.
    '''
    return spikegen.rate(input, num_steps=NUM_STEPS, gain=GAIN)

def encode_time(input):
    '''
    Encodes input data into spike trains using time-to-first-spike (latency) coding.

    Parameters:
    input (torch.Tensor): Input tensor to be encoded.

    Returns:
    torch.Tensor: Time-encoded spike train.
    '''
    return spikegen.latency(input, num_steps=NUM_STEPS, tau=TAU, threshold=THRESHOLD_LATENCY)

def encode_delta(input):
    '''
    Encodes input data using delta modulation, based on changes in rate-coded spikes.

    Parameters:
    input (torch.Tensor): Input tensor to be encoded.

    Returns:
    torch.Tensor: Delta-encoded spike train.
    '''
    return spikegen.delta(spikegen.rate(input, num_steps=NUM_STEPS, gain=GAIN), threshold=THRESHOLD_DELTA, off_spike=True)

## Gradio Application

In [10]:
# Load the models
model_rate = Neuromorphic_Net(images.shape, NUM_CLASSES)
model_delta = Neuromorphic_Net(images.shape, NUM_CLASSES)
model_time = Neuromorphic_Net(images.shape, NUM_CLASSES)

model_rate.load_state_dict(torch.load("neuromorphic_rate_model.pth", weights_only=False))
model_delta.load_state_dict(torch.load("neuromorphic_delta_model.pth", weights_only=False))
model_time.load_state_dict(torch.load("neuromorphic_temporal_model.pth", weights_only=False))

# Select a random image and label
def select_random_example():
    index = random.randint(0, len(testset) - 1)
    img, label = testset[index]
    return img, label

# Encode, visualize, and process the image
def encode_and_visualize(encoding_method):
    img, label = select_random_example()
    img = img.unsqueeze(0)  # Add batch dimension

    # Select encoding method and model
    if encoding_method == "Rate Encoding":
        spike_data = encode_rate(img)
        model = model_rate
    elif encoding_method == "Time-to-First-Spike Encoding":
        spike_data = encode_time(img)
        model = model_time
    elif encoding_method == "Delta Encoding":
        spike_data = encode_delta(img)
        model = model_delta

    # Visualize the spike train
    spike_data_sample = spike_data[:, 0, 0]
    fig, ax = plt.subplots()
    anim = splt.animator(spike_data_sample, fig, ax)

    # Save animation as a video file
    temp_video_path = os.path.join(tempfile.gettempdir(), "spike_animation.mp4")
    anim.save(temp_video_path, writer="ffmpeg", fps=10)
    plt.close(fig)

    # Get model output
    model.reset_states()
    outputs = []
    for step in spike_data:
        output_step = model(step)
        outputs.append(output_step)
    outputs = torch.stack(outputs)
    predicted_label = torch.argmax(outputs[-1], dim=1)

    return temp_video_path, f"Predicted: {predicted_label.item()}", f"Expected: {label}"

# Create Gradio interface
iface = gr.Interface(
    fn=encode_and_visualize,
    inputs=gr.Dropdown(["Rate Encoding", "Time-to-First-Spike Encoding", "Delta Encoding"], label="Choose Encoding Method"),
    outputs=[
        gr.Video(label="Spike Train Animation"),
        gr.Textbox(label="Model Prediction"),
        gr.Textbox(label="Expected Label"),
    ],
    title="🧠 Spiking Neural Network Visualization",
    description=(
        "This app demonstrates how a Spiking Neural Network (SNN) processes images using different encoding methods. "
        "Select an encoding method, and the app will convert a random test image into a spike train, "
        "visualize it, and use a trained SNN to classify the image. The model's predicted class will be compared with the expected label."
    ),
)

iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7109dcfdf0fb405263.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
